In [ ]:
import pandas as pd
import requests

Example of downloading data from BDL API for one territorial unit and many variables. 

Data on financial results of enterprises


|     |     |
| --- | --- |
| **Category**	| 	FINANCES OF ENTERPRISES (QUARTERLY DATA) (K43) |
| **Group**  | 	FINANCIAL RESULTS OF ENTERPRISES (G418) |
| **Subgroup** | Financial results of the PKD 2007 (P2695) |
| **Dimensions**	| Periods; Data coverage; Sections by PKD 2007; Years |
| **Last update** | 24/03/2021 |

Data in this category has, besides year and territorial unit, another 3 dimensions. For every combination of options in these 3 dimensions there is a separate variable with unique id. 

Parameters chosen for this example are: one territorial unit (Poland), years ... - ... , one option in period dimension (a year) and all available options in the two remaining dimensions.   



In [ ]:
# page for the whole subject 
first_page = requests.get('https://bdl.stat.gov.pl/api/v1/variables?subject-id=P2695&page-size=100', timeout=7)
first_page = first_page.json()

In [ ]:
# view data structure
first_page['results'][0]

{'id': 214413,
 'level': 3,
 'measureUnitId': 1,
 'measureUnitName': '-',
 'n1': 'I kwartał',
 'n2': 'przedsiębiorstwa objęte badaniem',
 'n3': 'Ogółem',
 'subjectId': 'P2695'}

In [ ]:
# option keys are named n1, n2, n3
# each combination of these options gives a variable with unique id

In [ ]:
last_link=first_page['links']['last']
last_link

'https://bdl.stat.gov.pl/api/v1/variables?subject-id=P2695&page-size=100&page=31'

In [ ]:
# get the last page number (31)
last_page_num=int(last_link[last_link.find("page=")+5:] )
last_page_num

31

In [ ]:
# crawl pages in this category to find variable id numbers
# select only the records (variables) where the period is a whole year ('n1' : 'rok')
var_list=[]
for page_num in range(0,last_page_num+1):
    page = requests.get('https://bdl.stat.gov.pl/api/v1/variables?subject-id=P2695&page-size=100&page='+str(page_num), timeout=7)
    page = page.json()
    for result in page['results']:
        if result['n1']=='rok':
            var_list.append(result)

In [ ]:
# 
len(var_list)

799

In [ ]:
var_list[0]

{'id': 216226,
 'level': 3,
 'measureUnitId': 1,
 'measureUnitName': '-',
 'n1': 'rok',
 'n2': 'przedsiębiorstwa objęte badaniem',
 'n3': 'Ogółem',
 'subjectId': 'P2695'}

In [ ]:
# n2 and n3 options
n2_set=set([x['n2'] for x in var_list])
n3_set=set([x['n3'] for x in var_list])

In [ ]:
print(len(n2_set), '*',len(n3_set),'=', len(n2_set)*len(n3_set))

47 * 17 = 799


In [ ]:
def get_var_id(n2,n3):
    for x in var_list:
        if x['n2']==n2 and x['n3']==n3:
            return x['id']
    return "variable not found"   

In [ ]:
# example
get_var_id('aktywa obrotowe', 'Budownictwo')

216680

In [ ]:
def get_n3(var_id):
    for x in var_list:
        if x['id']==var_id:
            return x['n3']

In [ ]:
# example
get_n3(216680)

'Budownictwo'

In [ ]:
all_dict={}

for n2 in n2_set:
    var_dict={}
    
    # only one page for every n2 (less than 100 records)
    url="https://bdl.stat.gov.pl/api/v1/data/by-unit/000000000000?page-size=100&" # one territorial unit
    for year in range(2011,2020+1):
        url+="year="+str(year)+"&"

    for n3 in n3_set:
        url+="var-id="+str(get_var_id(n2,n3))+"&"
    # many variables

    try:
        response = requests.get(url, timeout=7)
    except:
        print("request error")
        break
    page=response.json() 

    for result in page['results']:
        var_dict[get_n3(result['id'])]={x['year'] :x['val'] for x in result['values']}
    all_dict[n2]=var_dict

In [ ]:
len(all_dict)

47

In [ ]:
# dataframe example
pd.DataFrame(all_dict['aktywa obrotowe'])

,Opieka zdrowotna i pomoc społeczna,"Działalność związana z kulturą, rozrywką i rekreacją","Działalność profesjonalna, naukowa i techniczna",Działalność w zakresie usług administrowania i działalność wspierająca,"Wytwarzanie i zaopatrywanie w energię elektryczną, gaz, parę wodną, gorącą wodę i powietrze do układów klimatyzacyjnych",Informacja i komunikacja,Dostawa wody; gospodarowanie ściekami i odpadami oraz działalność związana z rekultywacją,Pozostała działalność usługowa,Działalność związana z obsługą rynku nieruchomości,Edukacja,Przetwórstwo przemysłowe,Górnictwo i wydobywanie,Działalność związana z zakwaterowaniem i usługami gastronomicznymi,"Handel hurtowy i detaliczny; naprawa pojazdów samochodowych, włączając motocykle",Ogółem,Budownictwo,Transport i gospodarka magazynowa
2011,1885448,1244164,16237606,12760484,57663041,30219628,8290427,599337,13185349,149310,317536901,31152305,2239432,158194701,739064765,61711315,25995317
2012,2282365,1373275,16828115,12537968,56449782,29843836,8272047,781187,13421074,208608,309408546,18521075,2079984,159229187,717571423,58346935,27987439
2013,3004267,1301495,18457776,16856945,54406239,28709855,9385443,965659,12521711,274280,320798920,18278262,2470506,164354414,737915241,55573520,30555949
2014,3357825,1236949,19528056,14736228,59797575,30077027,9753780,963889,12802938,361854,340459921,15139851,2376868,169354304,768733636,55846302,32940269
2015,4053053,1420188,19988765,17193396,57844417,34143204,9089526,1154375,13895836,381350,359107132,13959762,2354335,180371245,806896995,56488019,35452392
2016,4345042,1629313,21225106,20623863,60313562,32793657,10130888,1140292,15003942,247356,386783520,14200386,2423685,202580679,872218461,56043568,42733602
2017,4804080,2078387,22540920,24367771,59120763,35420090,10749184,986238,17064399,254624,417336272,18930545,3171149,229996102,956153126,61275294,48057308
2018,4810743,2252814,24365082,26519520,68858708,40720117,11198026,1081800,19913530,394457,432893086,19085564,3509916,243285949,1015096243,63404206,52802725
2019,5396336,2358664,26982233,30739862,77372511,42899530,12477531,1129624,22523749,501486,455283031,17069643,5298168,259004346,1083662856,68844833,55781309
2020,6602129,2397985,32811893,30121721,83575839,48709708,13906986,1229132,24719830,452602,475479410,18409560,4146732,268488330,1144847519,72647863,61147799


In [ ]:
# df for number of enterprises included in the survey
# with year as a variable column instead of index
df_ogolem=pd.DataFrame(all_dict['przedsiębiorstwa objęte badaniem'])
df_ogolem.index=df_ogolem.index.set_names(['rok'])
df_ogolem=df_ogolem.reset_index()
df_ogolem

,rok,Dostawa wody; gospodarowanie ściekami i odpadami oraz działalność związana z rekultywacją,"Działalność profesjonalna, naukowa i techniczna","Działalność związana z kulturą, rozrywką i rekreacją",Transport i gospodarka magazynowa,Działalność związana z zakwaterowaniem i usługami gastronomicznymi,Informacja i komunikacja,Górnictwo i wydobywanie,Działalność w zakresie usług administrowania i działalność wspierająca,Pozostała działalność usługowa,Przetwórstwo przemysłowe,Działalność związana z obsługą rynku nieruchomości,Budownictwo,Ogółem,Opieka zdrowotna i pomoc społeczna,Edukacja,"Handel hurtowy i detaliczny; naprawa pojazdów samochodowych, włączając motocykle","Wytwarzanie i zaopatrywanie w energię elektryczną, gaz, parę wodną, gorącą wodę i powietrze do układów klimatyzacyjnych"
0,2011,577,515,54,912,293,419,141,715,64,7267,499,1795,17346,359,25,3448,263
1,2012,590,552,52,955,297,456,143,767,62,7314,491,1773,17594,396,27,3465,254
2,2013,603,548,54,950,301,466,149,782,62,7212,474,1575,17239,440,48,3328,247
3,2014,603,559,54,959,292,481,146,817,57,7170,477,1413,17022,466,57,3226,245
4,2015,626,575,51,1006,292,492,143,878,61,7241,478,1354,17194,470,57,3233,237
5,2016,653,598,50,1025,303,517,136,909,56,7344,482,1315,17481,475,51,3345,222
6,2017,670,644,60,1115,323,556,131,973,55,7448,472,1233,17852,487,55,3407,223
7,2018,682,650,54,1116,319,576,132,975,63,7292,476,1224,17711,476,61,3395,220
8,2019,686,678,60,1136,327,612,132,979,56,7216,464,1202,17739,488,85,3398,220
9,2020,685,712,65,1149,329,639,127,984,55,7133,470,1211,17690,474,78,3369,210


In [ ]:
# unpivot df from wide to long format (creating df template for all variables)
df_ogolem_melt=pd.melt(df_ogolem,id_vars=['rok'],var_name='rodzaj', value_name='przedsiębiorstwa objęte badaniem')
df_ogolem_melt

,rok,rodzaj,przedsiębiorstwa objęte badaniem
0,2011,Dostawa wody; gospodarowanie ściekami i odpada...,577
1,2012,Dostawa wody; gospodarowanie ściekami i odpada...,590
2,2013,Dostawa wody; gospodarowanie ściekami i odpada...,603
3,2014,Dostawa wody; gospodarowanie ściekami i odpada...,603
4,2015,Dostawa wody; gospodarowanie ściekami i odpada...,626
...,...,...,...
165,2016,Wytwarzanie i zaopatrywanie w energię elektryc...,222
166,2017,Wytwarzanie i zaopatrywanie w energię elektryc...,223
167,2018,Wytwarzanie i zaopatrywanie w energię elektryc...,220
168,2019,Wytwarzanie i zaopatrywanie w energię elektryc...,220


In [ ]:
# join other variables to this df

all_df=df_ogolem_melt.copy() 

for variable_name in all_dict:
    if variable_name=="przedsiębiorstwa objęte badaniem":
        continue
    temp_df=pd.DataFrame(all_dict[variable_name])
    temp_df.index=temp_df.index.set_names(['rok'])
    temp_df=temp_df.reset_index()
    temp_df=pd.melt(temp_df,id_vars=['rok'],var_name='rodzaj', value_name=variable_name)
    all_df=pd.merge(all_df, temp_df, how='left', on=['rok','rodzaj'])

In [ ]:
# whole data in one table
all_df

,rok,rodzaj,przedsiębiorstwa objęte badaniem,przychody z całokształtu działalności (przychody ogółem),koszty finansowe,wynik zdarzeń nadzwyczajnych,wskaźnik płynności finansowej II stopnia,wynik finansowy na działalności gospodarczej,zysk netto,wskaźnik poziomu kosztów,pozostałe koszty operacyjne,należności krótkoterminowe z tytułu dostaw i usług,"wynik finansowy ze sprzedaży produktów, towarów i materiałów",inwestycje krótkoterminowe,wskaźnik rentowności obrotu netto,kredyty bankowe i pożyczki,zapasy,zobowiązania krótkoterminowe,wskaźnik rentowności obrotu brutto,należności krótkoterminowe,zobowiązania długoterminowe,przychody netto ze sprzedaży towarów i materiałów,przychody finansowe,obowiązkowe obciążenia wyniku finansowego brutto,zapasy - towary,zapasy - produkty gotowe,koszt sprzedanych produktów,zapasy - półprodukty i produkty w toku,"przychody netto ze sprzedaży produktów, towarów i materiałów",zobowiązania krótkoterminowe z tytułu dostaw i usług,udział liczby przedsiębiorstw wykazujących zysk netto w ogólnej liczbie przedsiębiorstw,udział przychodów przedsiębiorstw wykazujących zysk netto w przychodach z całokształtu działalności,strata brutto,przychody netto ze sprzedaży produktów,obowiązkowe obciążenia wyniku finansowego brutto - podatek dochodowy,wskaźnik rentowności sprzedaży brutto,koszty uzyskania przychodów z całokształtu działalności (koszty ogółem),zysk brutto,pozostałe przychody operacyjne,"zobowiązania krótkoterminowe z tytułu podatków, ceł, ubezpieczeń i innych świadczeń",wskaźnik płynności finansowej I stopnia,pozostałe przychody operacyjne - dotacje,strata netto,wynik finansowy netto,krótkoterminowe rozliczenia międzyokresowe,wynik finansowy brutto,zapasy - materiały,wartość sprzedanych towarów i materiałów,aktywa obrotowe
0,2011,Dostawa wody; gospodarowanie ściekami i odpada...,577,23461858,432227,3840.0,146.3,1346402.0,1165801,94.3,517854,2593385,1112748,3923378,4.4,1280937,744497,5028959,5.8,3434065,7746738,3701841,314526,315134,134008,64168,17924012,96070,22278123,1590056,84.4,88.8,127209,18576282,315134,5.0,22115456,1477451,869209,615353,78.0,164542,130693,1035108,188487,1350242,431629,3241363,8290427
1,2012,Dostawa wody; gospodarowanie ściekami i odpada...,590,23581851,440959,1721.0,151.6,1358285.0,1213815,94.2,606493,2522643,1037237,4022300,4.5,1409321,628192,4892222,5.8,3392950,7969006,2958204,398247,309824,100128,58008,18625057,69646,22213351,1495304,81.9,86.8,156928,19255147,309824,4.7,22223566,1516934,970253,634362,82.2,224600,163633,1050182,228605,1360006,391023,2551057,8272047
2,2013,Dostawa wody; gospodarowanie ściekami i odpada...,603,25417471,471336,-1155.0,154.2,2505457.0,2452816,90.1,558304,2757726,1173377,4827110,8.6,1480384,671763,5484138,9.9,3628114,8385365,2817678,307108,318000,132653,73169,19437103,44738,23055751,1612736,82.3,82.1,258569,20238073,318000,5.1,22912014,2762871,2054612,732605,88.0,286564,266514,2186302,258456,2504302,401069,2445271,9385443
3,2014,Dostawa wody; gospodarowanie ściekami i odpada...,603,25073304,414653,-1835.0,147.9,1666968.0,1495074,93.4,520978,2741862,1199854,5013315,5.2,1523108,699617,5932520,6.6,3760777,8483119,2604868,290459,352371,128251,78744,20256707,54087,23670559,1633723,84.7,85.1,182029,21065691,352371,5.1,23406336,1847162,1112286,757021,84.5,374547,182312,1312762,280071,1665133,400810,2213998,9753780
4,2015,Dostawa wody; gospodarowanie ściekami i odpada...,626,26021872,383177,-811.0,159.3,1965801.0,1731118,92.4,590686,2699250,1379456,4375318,6.1,1449935,649603,5122347,7.6,3787049,9548627,2539199,268154,368548,129560,81405,20904816,51679,24461664,1537960,86.3,86.9,134591,21922465,368548,5.6,24056071,2099581,1292054,718891,85.4,456673,134676,1596442,277556,1964990,344958,2177392,9089526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,2016,Wytwarzanie i zaopatrywanie w energię elektryc...,222,1

In [ ]:
# save data

In [ ]:
path='' 
# specify path

In [ ]:
all_df.to_csv(path+"finanse_przedsiębiorstw.csv", index=False)
#all_df.to_csv(path+"all_df.csv", index=False, encoding='utf-8')

In [ ]:
# additional
# handling measurement units

In [ ]:
all_df_u=all_df.copy()

In [ ]:
measure_unit_dict={}
for n2 in n2_set:
    for x in var_list:
        if x['n2']==n2:
            measure_unit_dict[n2]=x['measureUnitId']
            break

In [ ]:
def get_measure_unit(mu_id):
    #if mu_id==1:
    #    return 1
    if mu_id==5:    # tys. zł
        return 1000
    elif mu_id==50:    # %
        return 1/100
    else:
        return 1

for zmienna, mu_id in measure_unit_dict.items():
    all_df_u[zmienna]=all_df_u[zmienna]*get_measure_unit(mu_id)

In [ ]:
# view updated data
# all_df_u

In [ ]:
# save
all_df_u.to_csv(path+"finanse_przedsiębiorstw_u.csv", index=False)